In [1]:
import fnmatch
import cv2
import numpy as np
import string
import time

from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.utils import shuffle

In [43]:
import numpy as np
import string
from PIL import Image, ImageFont, ImageDraw
import random
import os
import cv2

n_samples = 10000
word_type = 'uppercase'

kernel = np.ones((2, 2), np.uint8)
kernel2 = np.ones((1, 1), np.uint8)
punclist = ',.?:;'

# specialss = "ĀŸÖЂЏÄāӱöђџä"

# Character sets to choose from
smallletters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюяЂђЃѓЄєІіЇїЈјЉљЊњЋћЌќЎўЏџҐґҒғҚқҮүҲҳҶҷӀӏӢӣӨөӮӯ!\#$%&'()*+-./:;<=>?@[\\]№_`{|}~ €₽0123456789"
    # specialss +
    # # Большие буквы
    # 'QWERTYUIOPASDFGHJKLZXCVBNMАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ' +
    # # Маленькие буквы
    # 'qwertyuiopasdfghjklzxcvbnmабвгдеёжзийклмнопрстуфхцчшщъыьэюя' +
    # # Цифры
    # '0123456789'

# smallletters = "qwertyuiopasdfghjklzxcvbnmабвгдеёжзийклмнопрстуфхцчшщъыьэюяєQWERTYUIOPASDFGHJKLZXCVBNMАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789"
capitalletters = smallletters
digits = string.digits

# Base background - только поддерживаемые форматы изображений
backfilelist = [f for f in os.listdir('background')
               if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
backgroud_list = []
for bn in backfilelist:
    img_path = 'background/' + bn
    img = cv2.imread(img_path, 0)
    if img is not None:
        backgroud_list.append(Image.fromarray(img))

# Different fonts to be used
fonts_list = ['fonts/' + f for f in os.listdir('fonts')
             if f.endswith(('.ttf', '.otf'))]

# Lengths of the words
word_lengths = list(range(1, 21))

# Font size
font_size = list(range(10, 30))

def random_brightness(img):
    return img

def dilation(img):
    img = np.array(img)
    return cv2.dilate(img, kernel2, iterations=1)

def erosion(img):
    img = np.array(img)
    return cv2.erode(img, kernel, iterations=1)

def fuse_gray(img):
    img = np.array(img)
    ht, wt = img.shape[:2]

    # Проверяем существование файла gray_back.jpg
    gray_back_path = 'gray_back.jpg'
    if not os.path.exists(gray_back_path):
        # Если файл не существует, создаем серый фон программно
        gray_back = np.full((ht, wt), 128, dtype=np.uint8)
    else:
        gray_back = cv2.imread(gray_back_path, 0)
        if gray_back is None:
            # Если не удалось прочитать файл, создаем серый фон программно
            gray_back = np.full((ht, wt), 128, dtype=np.uint8)
        else:
            gray_back = cv2.resize(gray_back, (wt, ht))

    return cv2.addWeighted(src1=img, alpha=0.8, src2=gray_back, beta=0.4, gamma=10)

def random_transformation(img):
    transformations = [fuse_gray, random_brightness, dilation, erosion]
    chosen_transformation = random.choice(transformations)
    result = chosen_transformation(np.array(img))
    return Image.fromarray(result)

# Создаем директории если не существуют
os.makedirs('images', exist_ok=True)

with open('annotation.txt', 'w', encoding='utf-8') as file:
    for file_counter in range(n_samples):
        if not backgroud_list:
            print("Нет фоновых изображений")
            break

        back_c = random.choice(backgroud_list).copy()
        start_cap = random.choice(capitalletters)
        filename = ''.join([random.choice(smallletters) for _ in range(random.choice([5, 6, 7, 8, 9, 10, 11]))])

        if not fonts_list:
            print("Нет доступных шрифтов")
            break

        try:
            font = ImageFont.truetype(random.choice(fonts_list), size=random.choice(font_size))
        except Exception as e:
            print(f"Ошибка загрузки шрифта: {e}")
            continue

        if word_type == 'lowercase':
            word = ''.join([random.choice(smallletters) for _ in range(random.choice(word_lengths))])
        elif word_type == 'uppercase':
            word = ''.join([random.choice(capitalletters) for _ in range(random.choice(word_lengths))])
        elif word_type == 'firstcapital':
            word = start_cap + ''.join([random.choice(smallletters) for _ in range(random.choice(word_lengths) - 1)])
        elif word_type == 'digits':
            word = ''.join([random.choice(digits) for _ in range(random.choice(word_lengths))])
        elif word_type == 'punctuation':
            word = ''.join([random.choice(smallletters) for _ in range(random.choice(word_lengths))]) + random.choice(punclist)
        else:
            raise Exception("Invalid word choice.")

        # Правильное использование getbbox()
        try:
            bbox = font.getbbox(word)
            w = bbox[2] - bbox[0]  # right - left
            h = bbox[3] - bbox[1]  # bottom - top
        except AttributeError:
            # Для старых версий PIL
            w, h = font.getsize(word)

        back_c = back_c.resize((w + 10, h + 10))  # Добавляем больше отступов
        draw = ImageDraw.Draw(back_c)
        draw.text((5, 5), text=word, font=font, fill='rgb(0,0,0)')  # Добавляем отступы для текста

        try:
            back_c = random_transformation(back_c)
        except Exception as e:
            print(f"Ошибка преобразования изображения: {e}")
            continue

        image_path = f'images/{file_counter}.jpg'
        back_c.save(image_path)

        # Print and save annotation
        print(f"Saved image: {image_path}")
        file.write(f"{file_counter}.jpg,{word}\n")

print("Датасет создан")
file.close()

char_list = smallletters

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
            pass

    return dig_lst

def find_dominant_color(image):
        #Resizing parameters
        width, height = 150,150
        image = image.resize((width, height),resample = 0)
        #Get colors from image object
        pixels = image.getcolors(width * height)
        #Sort them by count number(first element of tuple)
        sorted_pixels = sorted(pixels, key=lambda t: t[0])
        #Get the most frequent color
        dominant_color = sorted_pixels[-1][1]
        return dominant_color

def preprocess_img(img, imgSize):
    "put img into target img of size imgSize and normalize gray-values"

    # In case of black images with no text just use black image instead.
    if img is None:
        img = np.zeros([imgSize[1], imgSize[0]])
        print("Image None!")

    # create target image and copy sample image into it
    (wt, ht) = imgSize
    (h, w) = img.shape
    fx = w / wt
    fy = h / ht
    f = max(fx, fy)
    newSize = (max(min(wt, int(w / f)), 1),
               max(min(ht, int(h / f)), 1))  # scale according to f (result at least 1 and at most wt or ht)
    img = cv2.resize(img, newSize, interpolation=cv2.INTER_CUBIC)
    most_freq_pixel=find_dominant_color(Image.fromarray(img))
    target = np.ones([ht, wt]) * most_freq_pixel
    target[0:newSize[1], 0:newSize[0]] = img

    img = target

    return img


training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []

#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []

max_label_len = 0

annot=open('annotation.txt','r', encoding='utf-8').readlines()
imagenames=[]
txts=[]

for cnt in annot:
    filename,txt=cnt.split(',')[0],cnt.split(',')[1].split('\n')[0]
    imagenames.append(filename)
    txts.append(txt)

c = list(zip(imagenames, txts))

random.shuffle(c)

imagenames, txts = zip(*c)

for i in range(n_samples):
        img = cv2.imread('images/'+imagenames[i],0)

        img=preprocess_img(img,(128,32))
        img=np.expand_dims(img,axis=-1)
        img = img/255.
        txt = txts[i]

        # compute maximum length of the text
        if len(txt) > max_label_len:
            max_label_len = len(txt)


        # split the 150000 data into validation and training dataset as 10% and 90% respectively
        if i%10 == 0:
            valid_orig_txt.append(txt)
            valid_label_length.append(len(txt))
            valid_input_length.append(31)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(txt))
        else:
            orig_txt.append(txt)
            train_label_length.append(len(txt))
            train_input_length.append(31)
            training_img.append(img)
            training_txt.append(encode_to_labels(txt))
        print(f"{i} - success")

        # # break the loop if total data is 100000
        # if i == 100000:
        #     flag = 1
        #     break
        # # i+=1

# filepath = "variables.pickle"

# trainList = [training_img, training_txt, train_input_length, train_label_length, orig_txt]
# validList = [valid_img , valid_txt , valid_input_length, valid_label_length, valid_orig_txt]

# with open(filepath, 'wb') as file:
#     # Serialize and write the variable to the file
#     for i in range(len(trainList)):
#       pickle.dump(trainList[i], file)
#       pickle.dump(validList[i], file)

print("Конец составления")

<>:18: SyntaxWarning: invalid escape sequence '\#'
<>:18: SyntaxWarning: invalid escape sequence '\#'
C:\Users\mythy\AppData\Local\Temp\ipykernel_12516\2558528530.py:18: SyntaxWarning: invalid escape sequence '\#'
  smallletters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюяЂђЃѓЄєІіЇїЈјЉљЊњЋћЌќЎўЏџҐґҒғҚқҮүҲҳҶҷӀӏӢӣӨөӮӯ!\#$%&'()*+-./:;<=>?@[\\]№_`{|}~ €₽0123456789"


Saved image: images/0.jpg
Saved image: images/1.jpg
Saved image: images/2.jpg
Saved image: images/3.jpg
Saved image: images/4.jpg
Saved image: images/5.jpg
Saved image: images/6.jpg
Saved image: images/7.jpg
Saved image: images/8.jpg
Saved image: images/9.jpg
Saved image: images/10.jpg
Saved image: images/11.jpg
Saved image: images/12.jpg
Saved image: images/13.jpg
Saved image: images/14.jpg
Saved image: images/15.jpg
Saved image: images/16.jpg
Saved image: images/17.jpg
Saved image: images/18.jpg
Saved image: images/19.jpg
Saved image: images/20.jpg
Saved image: images/21.jpg
Saved image: images/22.jpg
Saved image: images/23.jpg
Saved image: images/24.jpg
Saved image: images/25.jpg
Saved image: images/26.jpg
Saved image: images/27.jpg
Saved image: images/28.jpg
Saved image: images/29.jpg
Saved image: images/30.jpg
Saved image: images/31.jpg
Saved image: images/32.jpg
Saved image: images/33.jpg
Saved image: images/34.jpg
Saved image: images/35.jpg
Saved image: images/36.jpg
Saved image

In [4]:
!pip install utils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13979 sha256=86031658855ec4ef236003458e45ad865a2f97d3071e4fe4d7fb0558016ed24d
  Stored in directory: c:\users\mythy\appdata\local\pip\cache\wheels\b6\a1\81\1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
import csv

def filter_allowed_chars(text, allowed_chars):
    return ''.join(c for c in text if c in allowed_chars)

def txt_to_csv(input_file, output_file):
    # Задаем разрешенные символы
    allowed_chars = set("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
                       "!#$%&'()*+,-./:;<=>?@[]№_`{|}~ €₽АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя"
                       "ЂђЃѓЄєІіЇїЈјЉљЊњЋћЌќЎўЏџҐґҒғҚқҮүҲҳҶҷӀӏӢӣӨөӮӯ")
    
    with open(input_file, 'r', encoding='utf-8') as txt_file:
        lines = txt_file.readlines()

    with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for line in lines:
            # Фильтруем символы и убираем переносы строк
            filtered_line = filter_allowed_chars(line.strip(), allowed_chars)
            # Записываем каждую строку как отдельную запись CSV
            writer.writerow([filtered_line])

if __name__ == "__main__":
    input_filename = "annotation.txt"
    output_filename = "labels.csv"
    txt_to_csv(input_filename, output_filename)

In [86]:
import pandas as pd

In [100]:
df = pd.read_csv("labels.csv", sep = ',', header = None, names = ['filenames'])
print(df.head(100))

                     filenames
0                  0.jpg,5Hвkг
1              1.jpg,ӢWвйДfsЯћ
2           2.jpg, ?;ГoJю'/ФЩЁ
3   3.jpg,}n&0oZаҚКZUBрѓ}cEnмS
4                  4.jpg,Ш)ҚLЯ
..                         ...
95         95.jpg,хш(0>jӮќЗoШС
96    96.jpg,qЫ5гтКРѓ9ЃAВ$ЯЬӮЇ
97                  97.jpg,<Њ!
98   98.jpg,ФИХ%x<)HL@ӣ!kеtг?ъ
99         99.jpg,ӮРјpzZћ+Ёго?

[100 rows x 1 columns]


In [101]:
df[['filenames', 'words']] = df['filenames'].str.split(',', expand=True)
df

,filenames,words
0,0.jpg,5Hвkг
1,1.jpg,ӢWвйДfsЯћ
2,2.jpg,?;ГoJю'/ФЩЁ
3,3.jpg,}n&0oZаҚКZUBрѓ}cEnмS
4,4.jpg,Ш)ҚLЯ
...,...,...
9995,9995.jpg,tӨВжя.З0ёМUkѓаӏjНdӢ
9996,9996.jpg,ғ}c1$Дѓ#ўC
9997,9997.jpg,qЕ
9998,9998.jpg,k_Ё


In [102]:
df

,filenames,words
0,0.jpg,5Hвkг
1,1.jpg,ӢWвйДfsЯћ
2,2.jpg,?;ГoJю'/ФЩЁ
3,3.jpg,}n&0oZаҚКZUBрѓ}cEnмS
4,4.jpg,Ш)ҚLЯ
...,...,...
9995,9995.jpg,tӨВжя.З0ёМUkѓаӏjНdӢ
9996,9996.jpg,ғ}c1$Дѓ#ўC
9997,9997.jpg,qЕ
9998,9998.jpg,k_Ё


In [103]:
df2 = df.iloc[8000:]
df2

,filenames,words
8000,8000.jpg,sJwнТЌk)ў(ӏ;f_
8001,8001.jpg,жX[мъaгМccлёЋaqіДҶU
8002,8002.jpg,rЈmјЌaщd№
8003,8003.jpg,іаїH@%Uу|қЊ>ПO
8004,8004.jpg,Lr}EmөNj*ъ|Ы5UўЏPa!
...,...,...
9995,9995.jpg,tӨВжя.З0ёМUkѓаӏjНdӢ
9996,9996.jpg,ғ}c1$Дѓ#ўC
9997,9997.jpg,qЕ
9998,9998.jpg,k_Ё


In [104]:
df = df.iloc[0:8000]

In [105]:
df

,filenames,words
0,0.jpg,5Hвkг
1,1.jpg,ӢWвйДfsЯћ
2,2.jpg,?;ГoJю'/ФЩЁ
3,3.jpg,}n&0oZаҚКZUBрѓ}cEnмS
4,4.jpg,Ш)ҚLЯ
...,...,...
7995,7995.jpg,эGv9mD@Қfша:т
7996,7996.jpg,ЖЁШiМ|љЋV
7997,7997.jpg,KйQЈ5пчirGLќyмйяЮ€
7998,7998.jpg,4*кРїG&Ша|оҳы+ђ>Ш


In [106]:
df.to_csv('train.csv', sep=',', index=False)


In [107]:
df2.to_csv('test.csv', sep = ',', index=False)